In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Lambda, Subtract
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dot


In [2]:
base_model = ResNet50(include_top=False, pooling='avg', weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

input_left = Input(shape=(200, 245, 3))
input_right = Input(shape=(200, 245, 3))

encoded_left = base_model(input_left)
encoded_right = base_model(input_right)

# Cosine similarity between the two encoded outputs
cosine_similarity = Dot(axes=1, normalize=True)([encoded_left, encoded_right])
prediction = Dense(1, activation='sigmoid')(cosine_similarity)

siamese_net = Model(inputs=[input_left, input_right], outputs=prediction)

optimizer = Adam(0.0001)
siamese_net.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [3]:
def load_image(img_path):
    img = image.load_img(img_path, target_size=(200, 245))
    img_array = image.img_to_array(img)
    return preprocess_input(img_array)

train_df = pd.read_csv('train.csv')

left_images = [load_image(os.path.join('train/left', fname + '.jpg')) for fname in train_df['left']]
right_images = [load_image(os.path.join('train/right', fname + '.jpg')) for fname in train_df['right']]
labels = np.ones(len(left_images))

left_images = np.array(left_images)
right_images = np.array(right_images)


In [4]:
siamese_net.fit([left_images, right_images], labels, batch_size=16, epochs=2)


Epoch 1/2
125/125 [==============================] - 779s 6s/step - loss: 0.6385 - accuracy: 1.0000
Epoch 2/2
125/125 [==============================] - 717s 6s/step - loss: 0.6208 - accuracy: 1.0000


In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)

test_df = pd.read_csv('test_candidates.csv')
test_df = test_df.head(5)
predictions = []

BATCH_SIZE = 5
NUM_BATCHES = 1  # 2000 left images divided by 100

for batch_idx in range(NUM_BATCHES):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = start_idx + BATCH_SIZE
    
    left_img_batch = np.array([load_image(os.path.join('test/left', fname + '.jpg')) for fname in test_df['left'][start_idx:end_idx]])
    left_img_batch = np.tile(left_img_batch, (20, 1, 1, 1, 1)).reshape(-1, 200, 245, 3)
    
    right_imgs = []
    for _, row in test_df[start_idx:end_idx].iterrows():
        right_imgs.extend([load_image(os.path.join('test/right', row[f'c{i}'] + '.jpg')) for i in range(20)])
    right_img_batch = np.array(right_imgs)
    
    batch_confidences = siamese_net.predict([left_img_batch, right_img_batch])
    batch_confidences = batch_confidences.reshape(BATCH_SIZE, 20)
    
    # Apply softmax
    batch_confidences = softmax(batch_confidences)
    
    for conf in batch_confidences:
        predictions.append(conf)


4/4 [==============================] - 9s 2s/step


In [ ]:
submission_df = pd.DataFrame(predictions, columns=[f'c{i}' for i in range(20)])
submission_df.insert(0, 'left', test_df['left'])
submission_df.to_csv('submission.csv', index=False)
